# Context Determination
As a suplement to the paper "Enhanced Online Grooming Detection Employing Context Determination and Message-Level Analysis" by Jake Street, Isibor Ihianle, Fuminiyi Olajide, and Ahmad Lotfi

To use this code you will need to have a list of one or more dictionaries. With these dictionaries containing your determinations (deters), author (authors) labels, and Online Grooming label (OGs) which denotes a valid AC context:
- Valid values for deters are: 1 <= n <= 0. (deters close to 1 represent an Adult message line prediction, whereas deters close to 0 represent a Child message line prediction)
- Valid values for authors are: 1 or 0. (these denote which author is linked to the determination and DO NOT indicate if the author is an Adult or Child actor)
- Valid values for OGs are: 1 or 0. (1 represents an adult actor, 0 represemts a child actor).

e.g. rOut = [{"deters":[0.9, 0.8, 0.1, 0.5, 0.2, 0.9], "authors":[1, 1, 0, 0, 0, 1], "OGs":[1, 1, 0, 1, 0, 1]}, {"deters":[...], "authors":[...], "OGs":[...]}]

In [ ]:
import pickle
import numpy as np
import scipy

In [ ]:
def binMLA(rOut, thresh): # This shall only be tested in A-C context at
    correct = 0
    incorrectA = 0
    incorrectC = 0
    ommit = 0
    for d in rOut:
        deters = list(d["deters"])
        auths = d["authors"]
        if "1" in auths or 1 in auths:
            for count in range(0, len(deters)):
                tempDeter = deters[count]
                tempAuth = auths[count]
                if tempDeter > (0.5 + thresh): # A determined
                    if str(tempAuth) == "1":
                        correct = correct + 1
                    else:
                        incorrectA = incorrectA + 1
                elif tempDeter < (0.5 - thresh):
                    if str(tempAuth) == "0":
                        correct = correct + 1
                    else:
                        incorrectC = incorrectC + 1
                else:
                    ommit = ommit + 1
    return [correct, incorrectA, incorrectC, ommit]


In [ ]:
def pCalc(l):
    isA = False
    std = np.std(l)
    mean = np.mean(l)
    u0 = 0.5
    sample = len(l)
    if mean > 0.5:
        isA = True
    T = ((mean - u0) / std) * np.sqrt(sample) # T score
    pval = scipy.stats.t.sf(np.abs(T), sample-1)*2 # P value
    return pval, isA


## Get Message Level Analysis Results

In [ ]:
thresh = 0.2 # t value to be used to generate results
path = "" # path to your pickle file (list of dictionaries as described above)

In [ ]:
with open(path, "rb") as resultPkl:
    rOut = pickle.load(resultPkl)
a = binMLA(rOut, thresh)
# print(a)
print(a[0]/(a[0]+a[1]+a[2])) # calc accuracy

## Get Context Determination Results

In [ ]:
with open(path, "rb") as resultPkl:
    rOut = pickle.load(resultPkl)

# Options

mesThreshList = [0, 0.1, 0.2, 0.3, 0.4] # t values to be tested
sigThreshList = [0.001, 0.01, 0.05] # ASTs to be tested

qFile = open("nameOfResultsFile.csv", "w")

for sigThresh in sigThreshList:
    for mesThresh in mesThreshList:
        correctACog = 0
        incorrectACog = 0
        correctNegative = 0
        incorrectNegative = 0
        for transcript in rOut:
            OGs = transcript["OGs"]
            Authors = transcript["authors"]
            deters = transcript["deters"]
            actorA = []
            actorB = []
            for count in range(0, len(Authors)):
                tempAuth = Authors[count]
                tempDeter = deters[count]
                if tempAuth == 1 and (tempDeter > 0.5 + mesThresh or tempDeter < 0.5 - mesThresh):
                    actorA.append(tempDeter)
                elif tempAuth == 0 and (tempDeter > 0.5 + mesThresh or tempDeter < 0.5 - mesThresh):
                    actorB.append(tempDeter)
            aPval, aIsA = pCalc(actorA)
            bPval, bIsA = pCalc(actorB)
            if 1 in OGs:
                if (aPval < sigThresh and bPval < sigThresh) and ((aIsA == True and bIsA == False) or (aIsA == False and bIsA == True)):
                    correctACog = correctACog + 1
                else:
                    incorrectNegative = incorrectNegative + 1
            else:
                if (aPval < sigThresh and bPval < sigThresh) and ((aIsA == True and bIsA == False) or (aIsA == False and bIsA == True)):
                    incorrectACog = incorrectACog + 1
                else:
                    correctNegative = correctNegative + 1
        lineToWrite = str(correctACog)+","+str(incorrectACog)+","+str(correctNegative)+","+str(incorrectNegative)+"\n"
        qFile.write(lineToWrite)
qFile.close()

